## GPT4v实战：API调用、使用场景实践

<div class="alert alert-warning">
<b>注意：</b>
<p>由于网络原因下面代码务必在自己的电脑上运行，才能看到界面</p>
</div>

## 步骤 1：安装所需依赖的包

确保你已经安装了 Python，并使用 pip 安装 Gradio, OpenAI, 以及Gemini所需依赖库：

#### 如果在本地电脑执行下面命令

```
pip  install gradio openai google-generativeai
```

## 如果在实验室执行下面命令

In [3]:
%pip install gradio==3.50  # 在实验室运行时执行的代码，本地不需要
%pip install -q -U google-generativeai

  Using cached gradio-3.50.0-py3-none-any.whl (20.3 MB)
  Using cached altair-5.2.0-py3-none-any.whl (996 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached gradio_client-0.6.1-py3-none-any.whl (299 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 121.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 37.7 MB/s eta 0:00:0000:0100:01
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 25.6 MB/s eta 0:00:00
  Using cached websockets-11.0.3-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
  Using cached contourpy-1.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (313 kB)

下面代码创建了一个 Gradio 用户界面，可以在文本框中输入问题，并上传最多三张图像。
这些输入将传递给 query_gpt4_vision 函数，该函数使用 OpenAI GPT-4 Vision 模型生成对问题的回答。

## 步骤 2：编写 Gradio 与 GPT-4 Vision 应用
在你的 Python 脚本中，编写 Gradio 应用。以下是一个例子，使用 GPT-4 Vision 模型：

In [1]:
import os
googleapi_key = os.getenv('GOOGLE_API_KEY')

In [12]:
import gradio as gr
from openai import OpenAI
import base64
from PIL import Image
import io, os
import google.generativeai as genai

# Function to encode the image to base64
def encode_image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Function to query GPT-4 Vision
def query_gpt4_vision(text, image1, image2, image3):
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    messages = [{"role": "user", "content": [{"type": "text", "text": text}]}]

    images = [image1, image2, image3]
    for image in images:
        if image is not None:
            base64_image = encode_image_to_base64(image)
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
            }
            messages[0]["content"].append(image_message)

    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=messages,
        max_tokens=1024,
    )
    return response.choices[0].message.content

# Function to query Gemini-Pro
def query_gemini_vision(text, image1, image2, image3):
    # Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
    #GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro-vision')

    images = [image1, image2, image3]
    query = [text]
    for image in images:
        if image is not None:
            query.append(image)
    response = model.generate_content(query, stream=False)
    response.resolve()
    
    return response.text

# 由于Gradio 2.0及以上版本的界面构建方式有所不同，这里使用blocks API来创建更复杂的UI
def main():
    with gr.Blocks() as demo:
        gr.Markdown("### 输入文本")
        input_text = gr.Textbox(lines=2, label="输入文本")
        input_images = [gr.Image(type="pil", label="Upload Image", tool="editor") for i in range(3)]
        output_gpt4 = gr.Textbox(label="GPT-4 输出")
        output_other_api = gr.Textbox(label="Gemini-Pro 输出")
        btn_gpt4 = gr.Button("调用GPT-4")
        btn_other_api = gr.Button("调用Gemini-Pro")

        btn_gpt4.click(fn=query_gpt4_vision, inputs=[input_text] + input_images, outputs=output_gpt4)
        btn_other_api.click(fn=query_gemini_vision, inputs=[input_text] + input_images, outputs=output_other_api)

    demo.launch(share=True)

if __name__ == "__main__":
    main()


Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://19c26f3cbf48f8cad2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 步骤 3：运行 Gradio 应用
保存并运行你的 Python 脚本。这将启动 Gradio 用户界面，并在终端中显示本地运行的 URL（通常是 http://127.0.0.1:7860）。访问该 URL 即可查看和使用你的 Gradio 应用。

<div class="alert alert-warning">
<b>建议：</b>
<li>确保你的机器上已经安装了必要的 Python 包。</li>
<li> 如果使用 OpenAI 模型，确保你已经设置了正确的 API 密钥。 </li>
</div>

## Examples

- 菜品价格预估：估算一下这桌子菜的价格，给出每道菜的市场价格。假设这是北京的一家中档餐厅
- figure理解：请生成这张图片的caption
- 网页设计：生成下面设计图对应的网站源码
- 视觉结合知识的推断：根据图中信息，猜测这是什么型号的GPU？请列出所有可能的GPU型号，并给出你的判断依据。猜测大致的生产年限

In [13]:
import gradio as gr
from openai import OpenAI
import base64
from PIL import Image
import io, os

# Function to encode the image to base64
def encode_image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Function to query GPT-4 Vision
def query_gpt4_vision(*inputs):
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    messages = [{"role": "user", "content": []}]

    for input_item in inputs:
        if isinstance(input_item, str):  # Text input
            messages[0]["content"].append({"type": "text", "text": input_item})
        elif isinstance(input_item, Image.Image):  # Image input
            base64_image = encode_image_to_base64(input_item)
            messages[0]["content"].append({
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
            })

    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=messages,
        max_tokens=1024,
    )
    return response.choices[0].message.content

# Function to query Gemini-Pro
def query_gemini_vision(*inputs):
    # Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
    #GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
    #print(GOOGLE_API_KEY)
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro-vision')

    query = []
    for item in inputs:
        if item is not None:
            query.append(item)
    response = model.generate_content(query, stream=False)
    response.resolve()
    
    return response.text

'''
    
# Dynamically generate input components
input_components = []
for i in range(2):  # Change this number to add more inputs
    input_components.append(gr.components.Textbox(lines=2, placeholder=f"Enter your text input {i+1}..."))
    input_components.append(gr.components.Image(type="pil", label=f"Upload Image {i+1}", tool="editor"))

iface = gr.Interface(
    fn=query_gpt4_vision,
    inputs=input_components,
    outputs=gr.components.Text(update=True), 
)

iface.launch(share=True)

'''
    
# 由于Gradio 2.0及以上版本的界面构建方式有所不同，这里使用blocks API来创建更复杂的UI
def main():
    with gr.Blocks() as demo:
        gr.Markdown("### 输入文本")
        #input_text = gr.Textbox(lines=2, label="输入文本")
        input_components = []
        for i in range(2):  # Change this number to add more inputs
            input_components.append(gr.Textbox(lines=2, placeholder=f"Enter your text input {i+1}..."))
            input_components.append(gr.Image(type="pil", label=f"Upload Image {i+1}", tool="editor"))
    
        #input_images = [gr.Image(type="pil", label="Upload Image", tool="editor") for i in range(3)]
        output_gpt4 = gr.Textbox(label="GPT-4 输出")
        output_other_api = gr.Textbox(label="Gemini-Pro 输出")
        btn_gpt4 = gr.Button("调用GPT-4")
        btn_other_api = gr.Button("调用Gemini-Pro")

        btn_gpt4.click(fn=query_gpt4_vision, inputs=input_components, outputs=output_gpt4)
        btn_other_api.click(fn=query_gemini_vision, inputs=input_components, outputs=output_other_api)

    demo.launch(share=True)

if __name__ == "__main__":
    main()


Running on local URL:  http://127.0.0.1:7867
Running on public URL: https://c4be06db92e0263fab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


- 具身智能场景：
  假设你是一个机器人，在厨房从事工作，你会执行的操作包括  靠近(物体坐标)， 抓取(物体坐标),  移动(开始坐标，结束坐标)，这里的坐标需要根据你的视觉系统来估计xy位置，以及深度信息z。人类会给你指令，你需要按照人类指令要求的完成对应的操作。比如，人类：把抽屉里的绿色包装袋拿出来。此时你看到的画面是这样的：
  请问接下来你该执行什么指令？只给出指令即可，但是需要包括具体的坐标信息（假设当前画面的长宽为单位1，使用你估计的深度信息以及xy偏移位置）
